In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
datapath = '/content/grdrive/NLP Assignment/'

Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
from nltk.tokenize import sent_tokenize

In [0]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
full_text = open('/content/gdrive/My Drive/NLP Assignment/jane_austen.txt', 'r', errors='ignore').read()

In [0]:
full_text = full_text.replace('\n', ' ').replace('\r', ' ')
#.replace('*** START: FULL LICENSE ***', " ")
#full_text = full_text.replace('SPEECH 1', ' ').replace('SPEECH 2', ' ').replace('SPEECH 3', ' ').replace('SPEECH 4', ' ').replace('SPEECH 5', ' ').replace('SPEECH 6', ' ').replace('SPEECH 7', ' ').replace('SPEECH 8', ' ').replace('SPEECH 9', ' ').replace('SPEECH 10', ' ')

In [0]:
import re

In [0]:
#full_text = re.sub(r'.*PERSUASION', '', full_text)
full_text = re.sub(r'Chapter\s[0-9]+', '', full_text)
full_text = re.sub(r'THE END.*', '', full_text)
full_text = full_text.lower()

In [79]:
print("CLASSICAL APPROACH...")

CLASSICAL APPROACH...


In [0]:
sent_tokenize_list = sent_tokenize(full_text)

In [0]:
for i in range(len(sent_tokenize_list)):
  sent_tokenize_list[i] = '# ' + sent_tokenize_list[i] + ' *'

In [0]:
train,test = train_test_split(sent_tokenize_list,test_size=0.2,random_state=40)

In [0]:
tokens=[]
for i in range(len(train)):
    tokens = tokens + nltk.word_tokenize(train[i])
tokens_uniq = np.array(tokens)
tokens_uniq=np.unique(tokens_uniq)
types = list(tokens_uniq)

In [84]:
print('Vocabulary Size: '+str(len(types)))

Vocabulary Size: 10599


In [0]:
from nltk.util import ngrams

In [0]:
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]

In [87]:
all_1grams = []
for sent in train:
  all_1grams += extract_ngrams(sent,1)

print("Total possible unigrams: "+ str(len(types)))
print("Number of unigrams that exist: "+ str(len(np.unique(all_1grams))))

Total possible unigrams: 10599
Number of unigrams that exist: 10599


In [88]:
all_2grams = []
for sent in train:
  all_2grams += extract_ngrams(sent,2)
print("Total possible bigrams: "+ str(len(types)**2))
print("Number of bigrams that exist: "+ str(len(np.unique(all_2grams))))

Total possible bigrams: 112338801
Number of bigrams that exist: 94691


In [89]:
all_3grams = []
for sent in train:
  all_3grams += extract_ngrams(sent,3)
print("Total possible trigrams: "+ str(len(types)**3))
print("Number of trigrams that exist: "+ str(len(np.unique(all_3grams))))

Total possible trigrams: 1190678951799
Number of trigrams that exist: 208528


In [90]:
all_4grams = []
for sent in train:
  all_4grams += extract_ngrams(sent,4)
print("Total possible quadgrams: "+ str(len(types)**4))
print("Number of quadgrams that exist: "+ str(len(np.unique(all_4grams))))

Total possible quadgrams: 12620006210117601
Number of quadgrams that exist: 263670


In [0]:
count_table_1gram = {}
for word in all_1grams:
  if word in count_table_1gram:
    count_table_1gram[word] += 1
  else:
    count_table_1gram[word] = 1

In [0]:
count_table_2gram = {}
for word in all_2grams:
  word_list = word.split(" ")
  context = " ".join(word_list[0:-1])
  if context in count_table_2gram: 
    if word_list[-1] in count_table_2gram[context]:
      count_table_2gram[context][word_list[-1]] += 1
    else:
      count_table_2gram[context][word_list[-1]] = 1
  else:
    count_table_2gram[context]={}
    count_table_2gram[context][word_list[-1]] = 1

In [0]:
count_table_3gram = {}
for word in all_3grams:
  word_list = word.split(" ")
  context = " ".join(word_list[0:-1])
  if context in count_table_3gram: 
    if word_list[-1] in count_table_3gram[context]:
      count_table_3gram[context][word_list[-1]] += 1
    else:
      count_table_3gram[context][word_list[-1]] = 1
  else:
    count_table_3gram[context]={}
    count_table_3gram[context][word_list[-1]] = 1

In [0]:
count_table_4gram = {}
for word in all_4grams:
  word_list = word.split(" ")
  context = " ".join(word_list[0:-1])
  if context in count_table_4gram: 
    if word_list[-1] in count_table_4gram[context]:
      count_table_4gram[context][word_list[-1]] += 1
    else:
      count_table_4gram[context][word_list[-1]] = 1
  else:
    count_table_4gram[context]={}
    count_table_4gram[context][word_list[-1]] = 1

In [0]:
def MLE(count_table, context, word):
  try:
    if context == '':
      return float(count_table[word]/sum(count_table.values()))
    else:
      count_ngram = count_table[context][word]
      count_context = sum(count_table[context].values())
      return float(count_ngram/count_context)
  except:
    return 0

In [0]:
count_tables = [count_table_1gram, count_table_2gram, count_table_3gram, count_table_4gram]

In [0]:
def Generator(num, sent_count):
  for i in range(sent_count):
    new_sent = ["#"]    
    while (len(new_sent) < num-1):
      chosen = '' 
      random = np.random.multinomial(1, [MLE(count_tables[1], " ".join(new_sent[-1:]), word) for word in np.unique(all_1grams)], size=1)
      for i in range(len(random[0])):
        if random[0][i] == 1:
          chosen = np.unique(all_1grams)[i]
      new_sent.append(chosen)
    chosen = ''
    words_in_sent = 0  
    while (chosen!='*' and words_in_sent<25):
      if num==1:
        random = np.random.multinomial(1, [MLE(count_tables[0], "", word) for word in np.unique(all_1grams)], size=1)
      else:
        random = np.random.multinomial(1, [MLE(count_tables[num-1], " ".join(new_sent[-(num-1):]), word) for word in np.unique(all_1grams)], size=1)
      for i in range(len(random[0])):
        if random[0][i] == 1:
          chosen = np.unique(all_1grams)[i]
      new_sent.append(chosen)
      words_in_sent += 1
    print(" ".join(new_sent[1:-1]))

In [0]:
test_tokens=[]
for i in range(len(test)):
    test_tokens = test_tokens + nltk.word_tokenize(test[i])
#tokens_uniq = np.array(tokens)
#tokens_uniq=np.unique(tokens_uniq)
#types = list(tokens_uniq)

In [0]:
from statistics import mean

In [0]:
def prob_calc(sent_list, num):
  prob = [];
  for i in range(len(sent_list)):
    words_in_sent = nltk.word_tokenize(sent_list[i])
    for k in range(len(words_in_sent)):
      if num == 1:
        mle = MLE(count_tables[0], "", words_in_sent[k])
      else:
        if k < num-1:
          mle = MLE(count_tables[0], "", words_in_sent[k])
        else: 
          mle = MLE(count_tables[num-1], " ".join(words_in_sent[k-(num-1):k]), word)
      if mle != 0:
        prob.append(np.log(mle))
  return np.exp(mean(prob))
  #return np.exp(prob)
  

In [0]:
def pplx_calc(sent_list, num):
  prob = prob_calc(sent_list, num)
  pplx = prob ** (-1/len(test_tokens))
  return pplx

In [102]:
print("Perplexity of the different Language Models on the test data: \n")
print("Unigram Model: " + str(pplx_calc(test,1)))
print("Bigram Model: " + str(pplx_calc(test,2)))
print("Trigram Model: " + str(pplx_calc(test,3)))
print("Quadgram Model: " + str(pplx_calc(test,4)))

Perplexity of the different Language Models on the test data: 

Unigram Model: 1.0000729882878776
Bigram Model: 1.0000412541697576
Trigram Model: 1.0000545737006532
Quadgram Model: 1.0000629581935199


In [103]:
print("Sentences Generated by Unigram LM: \n")
Generator(1, 5)
print("\n\nComments: Most of the sentences are grammatically incorrect and do not make any sense because they are just series of words chosen at random. Also, the sentences are intersparsed with random occurences of punctuation marks because the model is not taking context into account.")

Sentences Generated by Unigram LM: 

entirely does joined than the of , distressing was support '' well out # by to had he using get
his , few itself learnt to
morland if observe it and be modern average `` #
not amongst . fears to of this to serve # not morland norris consolation `` his returned steady a so i of the dinner
commodious with feelings west , bertrams ,


Comments: Most of the sentences are grammatically incorrect and do not make any sense because they are just series of words chosen at random. Also, the sentences are intersparsed with random occurences of punctuation marks because the model is not taking context into account.


In [104]:
print("Sentences Generated by Bigram LM:\n")
Generator(2,5)
print("\n\nComments: Although the sentences do not have random occurences of punctuation marks like in the case of unigram model, the sentences still do not make much sense.")

Sentences Generated by Bigram LM:

his disorder increased pain , and used to her , a repetition ; and that room for out of tenderness toward herself in the
she talked a future war with a frigate , though she deserved ; and even when louisa and with perfect model of such a
where a home at school while anne could , so little drawing-room , and do i. we should allow myself ; and i had
we respect .
captain wentworth , perhaps i am not rather have passed between them into the evening , dear !


Comments: Although the sentences do not have random occurences of punctuation marks like in the case of unigram model, the sentences still do not make much sense.


In [105]:
print("Sentences Generated by Trigram LM:\n")
Generator(3,5)
print("\n\nComments: Some parts of the sentences make more sense because the model is capturing more context than before.")

Sentences Generated by Trigram LM:

but they could not forbear , in a state of utter barbarism , to interfere with the strafford family .
perhaps they ought to have plenty of seeing them .
sometimes it appeared that blaize castle remained her only concern of higher interest .
she was in the garden with mackenzie , trying to bury some of the hand ; but the evil would have expected anything rather than
she was sure they would be turned , repeating her conviction of his coming might have warned her , which fills up the room ,


Comments: Some parts of the sentences make more sense because the model is capturing more context than before.


In [106]:
print("Sentences Generated by Quadgram LM:\n")
Generator(4,5)
print("\n\nComments: The sentences generated are very similar to the sentences in the training set because a lot of the context is being considered.")

Sentences Generated by Quadgram LM:

a few days , he could not have been introduced to each other with a considerable portion of apparent indifference and calmness ; but the only
there indulged zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous zealous
i hope she does not do what her aunt and to herself .
i would not go for the sake of being travelling at the same time from susan 's .
i did not feel equal to it .


Comments: The sentences generated are very similar to the sentences in the training set because a lot of the context is being considered.


In [107]:
print("NEURAL APPROACH...")

NEURAL APPROACH...


In [0]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import SimpleRNN
from keras.layers import Embedding
from keras.layers import RNN
from keras.utils import np_utils
import keras.preprocessing.text as ktp

In [0]:
words_list = list(ktp.text_to_word_sequence(full_text, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'', lower=True, split=' '))
words = sorted(list(set(words_list)))

In [0]:
n_to_word = {n:word for n, word in enumerate(words)}
word_to_n = {word:n for n, word in enumerate(words)}

In [15]:
vocab_size = len(words)
print('Number of unique words: ', vocab_size)

Number of unique words:  10811


In [0]:
X = []   # extracted sequences ---- train array
Y = []   # follow up word for each sequence in X ---- target array
length = len(words_list)
seq_length = 7

for i in range(0, length - seq_length, 1):
    sequence = words_list[i:i + seq_length]
    label = words_list[i + seq_length]
    X.append([word_to_n[w] for w in sequence])
    Y.append(word_to_n[label])
    
#print('Number of extracted sequences:', len(X))

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [0]:
X_mod = np.reshape(X_train, (len(X_train), seq_length, 1))
X_mod = X_mod / float(len(words))
Y_mod = np_utils.to_categorical(Y_train)

In [19]:
model = Sequential()
#model.add(Embedding(vocab_size, 100))
model.add(SimpleRNN(100, return_sequences = True))
#, input_shape=(X_mod.shape[1], X_mod.shape[2]), return_sequences=True
model.add(Dropout(0.2))
model.add(SimpleRNN(100))
model.add(Dropout(0.2))
model.add(Dense(len(words), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

In [30]:
model.fit(X_mod, Y_mod, epochs=10, batch_size=128, callbacks = [reduce_lr])

Epoch 1/10
259205/259205 [==============================] - 54s 209us/step - loss: 6.2028 - acc: 0.0422
Epoch 2/10
259205/259205 [==============================] - 54s 209us/step - loss: 6.1853 - acc: 0.0424
Epoch 3/10
259205/259205 [==============================] - 54s 209us/step - loss: 6.1671 - acc: 0.0427
Epoch 4/10
259205/259205 [==============================] - 54s 210us/step - loss: 6.1544 - acc: 0.0423
Epoch 5/10
259205/259205 [==============================] - 55s 210us/step - loss: 6.1393 - acc: 0.0427
Epoch 6/10
259205/259205 [==============================] - 55s 213us/step - loss: 6.1234 - acc: 0.0424
Epoch 7/10
259205/259205 [==============================] - 55s 211us/step - loss: 6.1126 - acc: 0.0425
Epoch 8/10
259205/259205 [==============================] - 55s 211us/step - loss: 6.0977 - acc: 0.0419
Epoch 9/10
259205/259205 [==============================] - 55s 213us/step - loss: 6.0854 - acc: 0.0425
Epoch 10/10
259205/259205 [==============================] - 55s

In [0]:
X_modtest = np.reshape(X_test, (len(X_test), seq_length, 1))
X_modtest = X_modtest / float(len(words))
Y_modtest = np_utils.to_categorical(Y_test, num_classes = 10811)

In [0]:
def sample(preds, temperature=1.0):
  # helper function to sample an index from a probability array
  preds
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  #print(preds[0,:])
  probas = np.random.multinomial(1, preds, 1)
  probas
  return np.argmax(probas)


In [0]:
def make_a_sent(model):
  start = np.random.randint(0, len(X)-1) # or generate random start
  words_mapped = list(X[start])
  sent = [n_to_word[value] for value in X[start]]
  # generating words
  started = 1
  pred_index = 0
  for i in range(20):
    x = np.reshape(words_mapped,(1,len(words_mapped), 1))
    x = x / float(len(words))
    pred_index = sample(model.predict(x, verbose=0)[0,:])
    #print(words_mapped)
    sent.append(n_to_word[pred_index])

    words_mapped.append(pred_index)  # add the predicted word to the end
    words_mapped = words_mapped[1:len(words_mapped)] # shift the string one word forward by removing pos. 0
  output = " ".join(sent)
  return output

In [35]:
print("Sentences Generated by Vanilla RNN:\n")
print(make_a_sent(model))
print(make_a_sent(model))
print(make_a_sent(model))
print(make_a_sent(model))
print(make_a_sent(model))
print("\n\nComments: The sentences do not make sense. They are not readable.")

Sentences Generated by Vanilla RNN:

go her cousin was safe on the at first a passed with creature sensible rest which at to man highly wonder time she not anything till first
his sister and whispered my dear creature entered its her he own if that to came pleasure something she sight being least not seen looked its for
of some grandeur in any particular style he common them it yourself them it a doubt fall sort probably you ragged i dear of down agreeable i
been governed by motives of selfishness and one cash the give if still were she persuade the you reigned catherine them matter week mischief done we house
say that he believed her to have she with will when longed talked you undoubtedly favourable a having he had you making of it that learn and


Comments: The sentences do not make sense. They are not readable.


In [0]:
from keras.layers import LSTM

In [0]:
model_lstm = Sequential()
#model.add(Embedding(vocab_size, 100))
model_lstm.add(LSTM(100, input_shape=(X_mod.shape[1], X_mod.shape[2]), return_sequences=True))
#
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(100))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(Y_mod.shape[1], activation='softmax'))
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
model_lstm.fit(X_mod, Y_mod, epochs=10, batch_size=128, callbacks = [reduce_lr])

Epoch 1/10
259205/259205 [==============================] - 107s 414us/step - loss: 6.5969 - acc: 0.0369
Epoch 2/10
259205/259205 [==============================] - 104s 401us/step - loss: 6.4554 - acc: 0.0384
Epoch 3/10
259205/259205 [==============================] - 103s 398us/step - loss: 6.4404 - acc: 0.0387
Epoch 4/10
259205/259205 [==============================] - 103s 397us/step - loss: 6.4322 - acc: 0.0388
Epoch 5/10
259205/259205 [==============================] - 102s 395us/step - loss: 6.4282 - acc: 0.0391
Epoch 6/10
259205/259205 [==============================] - 103s 397us/step - loss: 6.4243 - acc: 0.0390
Epoch 7/10
259205/259205 [==============================] - 103s 397us/step - loss: 6.4215 - acc: 0.0391
Epoch 8/10
259205/259205 [==============================] - 103s 396us/step - loss: 6.4178 - acc: 0.0390
Epoch 9/10
259205/259205 [==============================] - 103s 396us/step - loss: 6.4143 - acc: 0.0392
Epoch 10/10
259205/259205 [============================

In [39]:
print("Sentences Generated by RNN with LSTM:\n")
print(make_a_sent(model_lstm))
print(make_a_sent(model_lstm))
print(make_a_sent(model_lstm))
print(make_a_sent(model_lstm))
print(make_a_sent(model_lstm))
print("\n\nComments: The sentences are not readable, that is, do not make sense. But they contain some phrases (group of words) that are meaningful.")

Sentences Generated by RNN with LSTM:

you very particularly asked even if you only her will in that and as the any you must she has my rejoice us as anxiety locked that
a scheme among such a party and that brought proudly and of together and come part making with going time left in visited a but many it
determined they should not say i shut do but their did him spoke of been of build does his down in manner do nothing s way admit
them and felt for them with a a he suffer been you i young especially upon to hand joined husband nothing preferred in i amazed resumed her
enclosing two more one from the secretary and to her mansfield of as is judgment heard plans that not had among without and through morland the wind


Comments: The sentences are not readable, that is, do not make sense. But they contain some phrases (group of words) that are meaningful.


In [0]:
def prob_sent(X_test, Y_test, model):
  tot_prob = []
  i=0
  while (i<len(X_test)):
    prob_sent = 0
    for j in range(20):
      try:
        words_mapped = list(X_test[i])
        x = np.reshape(words_mapped,(1,len(words_mapped), 1))
        x = x / float(len(words))
        y = model.predict(x,verbose=0)[0,:]
        pred = np.argmax(Y_test[i])
        #print(y[pred])
        if y[pred]!=0:
          prob = np.log(y[pred])
          prob_sent += prob
        i+=1
      except:
        continue
    tot_prob.append(np.exp(prob_sent))
  return np.mean(tot_prob)





In [0]:
def ppx(X_test, Y_test, model):
  prob = prob_sent(X_test, Y_test, model)
  N = len(Y_test)
  return prob**(-1/N)

In [0]:
print("Perplexity of different neural models on the test data set:\n")
print("Vanilla RNN: "+ str(ppx(X_modtest,Y_modtest, model)))
print("LSTM: "+ str(ppx(X_modtest, Y_modtest, model_lstm)))

Perplexity of different neural models on the test data set:



In [42]:
print("The text generated by n-gram model, especially in case of trigrams and quadgrams is more readable than the neural models. Neural models attemot to capture more context during training and prediction. However, at 10 epochs, they are not able to make good predictions. They could perform better with more number of epochs. I could not test that due to time constraint. However, too many epochs can also lead to overfitting. N-gram models are taking a smaller context into consideration but they are choosing from actual n-gram occurences. So, they are making more meaningful predictions in this case of text generation.")

The text generated by n-gram model, especially in case of trigrams and quadgrams is more readable than the neural models. Neural models attemot to capture more context during training and prediction. However, at 10 epochs, they are not able to make good predictions. They could perform better with more number of epochs. I could not test that due to time constraint. However, too many epochs can also lead to overfitting. N-gram models are taking a smaller context into consideration but they are choosing from actual n-gram occurences. So, they are making more meaningful predictions in this case of text generation.
